In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, when, lower
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

21/10/28 23:18:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Carga del dataset

In [3]:
schema = StructType([StructField("periodo", IntegerType(), True),
                     StructField("id_cli", LongType(), True),
                     StructField("fecha_nacimiento", LongType(), True),
                     StructField("edad", DoubleType(), True),
                     StructField("genero", StringType(), True),
                     StructField("estado_civil", StringType(), True),
                     StructField("nivel_academico", StringType(), True),
                     StructField("profesion", StringType(), True),
                     StructField("ocupacion", StringType(), True),
                     StructField("tipo_vivienda", StringType(), True),
                     StructField("ult_actual", LongType(), True),
                     StructField("categoria", ShortType(), True),
                     StructField("codigo_ciiu", LongType(), True),
                     StructField("ind_mora_vigente", StringType(), True),
                     StructField("cartera_castigada", StringType(), True),
                     StructField("ciudad_residencia", StringType(), True),
                     StructField("departamento_residencia", StringType(), True),
                     StructField("ciudad_laboral", StringType(), True),
                     StructField("departamento_laboral", StringType(), True),
                     StructField("rechazo_credito", StringType(), True),
                     StructField("mora_max", LongType(), True),
                     StructField("cant_moras_30_ult_12_meses", LongType(), True),
                     StructField("cant_moras_60_ult_12_meses", LongType(), True),
                     StructField("cant_moras_90_ult_12_meses", LongType(), True),
                     StructField("cupo_total_tc", DoubleType(), True),
                     StructField("tenencia_tc", StringType(), True),
                     StructField("cuota_tc_bancolombia", DoubleType(), True),
                     StructField("tiene_consumo", StringType(), True),
                     StructField("tiene_crediagil", StringType(), True),
                     StructField("nro_tot_cuentas", LongType(), True),
                     StructField("ctas_activas", LongType(), True),
                     StructField("tiene_ctas_activas", StringType(), True),
                     StructField("ctas_embargadas", LongType(), True),
                     StructField("tiene_ctas_embargadas", StringType(), True),
                     StructField("pension_fopep", StringType(), True),
                     StructField("cuota_cred_hipot", DoubleType(), True),
                     StructField("tiene_cred_hipo_1", StringType(), True),
                     StructField("tiene_cred_hipo_2", StringType(), True),
                     StructField("mediana_nom3", DoubleType(), True),
                     StructField("mediana_pen3", DoubleType(), True),
                     StructField("ingreso_nompen", DoubleType(), True),
                     StructField("cat_ingreso", StringType(), True),
                     StructField("ingreso_final", DoubleType(), True),
                     StructField("cant_mora_30_tdc_ult_3m_sf", DoubleType(), True),
                     StructField("cant_mora_30_consum_ult_3m_sf", DoubleType(), True),
                     StructField("cuota_de_vivienda", DoubleType(), True),
                     StructField("cuota_de_consumo", DoubleType(), True),
                     StructField("cuota_rotativos", DoubleType(), True),
                     StructField("cuota_tarjeta_de_credito", DoubleType(), True),
                     StructField("cuota_de_sector_solidario", DoubleType(), True),
                     StructField("cuota_sector_real_comercio", DoubleType(), True),
                     StructField("cupo_tc_mdo", DoubleType(), True),
                     StructField("saldo_prom3_tdc_mdo", DoubleType(), True),
                     StructField("cuota_tc_mdo", DoubleType(), True),
                     StructField("saldo_no_rot_mdo", DoubleType(), True),
                     StructField("cuota_libranza_sf", DoubleType(), True),
                     StructField("cant_oblig_tot_sf", DoubleType(), True),
                     StructField("cant_cast_ult_12m_sr", DoubleType(), True),
                     StructField("ind", DoubleType(), True),
                     StructField("rep_calif_cred", StringType(), True),
                     StructField("pol_centr_ext", DoubleType(), True),
                     StructField("convenio_lib", StringType(), True),
                     StructField("ingreso_nomina", DoubleType(), True),
                     StructField("ingreso_seguridad_social", DoubleType(), True),
                     StructField("gasto_familiar", DoubleType(), True),
                    ])


In [4]:
df = sqlCtx.read.format("csv") \
           .option("header", False) \
           .option("delimiter", ";") \
           .schema(schema) \
           .load("Dataton_train_semicolon.csv")

In [5]:
sampling = df.sample(False, 0.025, 95757)

In [6]:
#sampling.repartition(1).write.mode('overwrite').parquet('tmp/sampling')
sampling.repartition(1).write.csv("sample.csv")

21/10/28 23:18:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Se transformará el dataset en formato parquet para un mejor uso del mismo

In [7]:
#df.repartition(1).write.mode('overwrite').parquet('tmp/dataton')